In [62]:
import pandas as pd
from datetime import datetime, timedelta, time


In [27]:
# Import Work Status Per Time table that will be used for prioritizing registors based on remaining work time left

work_status_df= pd.read_excel(r"Intermediate_Results\sample_work_status_per_time_table.xlsx")

# Select only required columns
cols= ['Start_time', 'End_time', 'Employee', 'Remaining_hours_left']
work_status_df= work_status_df[cols] 

# Convert to datetime.time
def to_time(x):
    return datetime.strptime(x, "%H:%M:%S").time()
work_status_df['Start_time'] = work_status_df['Start_time'].apply(to_time)
work_status_df['End_time'] = work_status_df['End_time'].apply(to_time)

work_status_df.head()

,Start_time,End_time,Employee,Remaining_hours_left
0,08:30:00,09:00:00,Andrew (AJ) # Bowlen,5.0
1,08:30:00,09:00:00,Analuisa Flores Teran,2.0
2,08:30:00,09:00:00,Finn Broderick,2.0
3,08:30:00,09:00:00,Ricardo Gomez,3.0
4,08:30:00,09:00:00,Solymar Kneale,2.5


In [28]:
work_status_df.iloc[0,0]

datetime.time(8, 30)

In [31]:
# Import Shift Count Rquirement table that will be used to know how registors have to be assigned

shift_req_df= pd.read_excel(r"Intermediate_Results\Sample_Shift_Count_Rquirement_table.xlsx")

# Select only required columns
cols= ['From_Time', 'To_Time', 'Reg_Up_Needed', 'Reg_Down_Needed']
shift_req_df= shift_req_df[cols] 

# Convert to datetime.time
def to_time(x):
    return datetime.strptime(x, "%H:%M:%S").time()
shift_req_df['From_Time'] = shift_req_df['From_Time'].apply(to_time)
shift_req_df['To_Time'] = shift_req_df['To_Time'].apply(to_time)

shift_req_df.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed
0,07:30:00,08:00:00,3,3
1,08:00:00,08:30:00,3,3
2,08:30:00,09:00:00,3,3
3,09:00:00,09:30:00,3,3
4,09:30:00,10:00:00,3,3


In [71]:
# Import Greeter Assignment table that will be used to know how many registers were moved to greeter

greeter_assignment= pd.read_excel(r"Intermediate_Results\sample_greeter_assignment_table.xlsx")

# Select only required columns
cols= ['From_Time', 'To_Time', 'Employee_Assigned_Upstairs', 'Employee_Assigned_Downstairs']
greeter_assignment= greeter_assignment[cols] 

# Convert to datetime.time
def to_time(x):
    return datetime.strptime(x, "%H:%M:%S").time()
greeter_assignment['From_Time'] = greeter_assignment['From_Time'].apply(to_time)
greeter_assignment['To_Time'] = greeter_assignment['To_Time'].apply(to_time)

greeter_assignment.head()


,From_Time,To_Time,Employee_Assigned_Upstairs,Employee_Assigned_Downstairs
0,07:30:00,08:00:00,NaN,NaN
1,08:00:00,08:30:00,NaN,NaN
2,08:30:00,09:00:00,['Analuisa Flores Teran'],['Mariana Cepeda']
3,09:00:00,09:30:00,Dev Mahajan,['Joaquin Salinas']
4,09:30:00,10:00:00,['Ricardo Gomez'],['Lauren* Koski']


In [101]:
# Function to convert the string of list to list in the greeter assignment table
def extract_list_from_string(string):
    return string.replace("'",'').replace("[",'').replace("]",'').split(",")

In [ ]:

# Prepare intial variables
current_RUs= []
current_RDs= []
store_open_time= time(8, 30) #shift_req_df['From_Time'].min()
store_close_time= shift_req_df['From_Time'].max()
time_delta= timedelta(minutes=30)

curr_time= store_open_time

while (curr_time < store_close_time):
    curr_time_plus30= (datetime.combine(datetime.today(), curr_time) + time_delta).time()
    print(f"Allocating registers between {curr_time} to {curr_time_plus30} ...")

    # Working Employees
    all_working_emp_list= work_status_df[(work_status_df['Start_time']== curr_time) & (work_status_df['End_time']== curr_time_plus30)]['Employee'].to_list()
    greeter_emp_list= extract_list_from_string(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Employee_Assigned_Upstairs'].to_list()[0]) + extract_list_from_string(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Employee_Assigned_Downstairs'].to_list()[0])

    # Baseed on the workring and greeter list, calculate the retined register list
    retained_RUs=[]
    retained_RDs= []

    for emp in current_RUs:
        if (emp not in all_working_emp_list):
            # EMP shift got over
            continue
        if (emp in greeter_emp_list):
            # EMP moved to greeter
            continue
        else:
            retained_RUs.append(emp)
    
    for emp in current_RDs:
        if (emp not in all_working_emp_list):
            # EMP shift got over
            continue
        if (emp in greeter_emp_list):
            # EMP moved to greeter
            continue
        else:
            retained_RDs.append(emp)

    # Find how many new registers are needed
    needed_RU_count, needed_RD_count= shift_req_df[(shift_req_df['From_Time']== curr_time) & (shift_req_df['To_Time']== curr_time_plus30)][['Reg_Up_Needed','Reg_Down_Needed']].values[0]
    RU_retained_count= len(retained_RUs)
    RD_retained_count= len(retained_RDs)
    new_RU_needed_count= needed_RU_count-RU_retained_count
    new_RD_needed_count= needed_RD_count-RD_retained_count

    # Create the prriority table. Use the same table to assign both RU and RD
    priority_table= work_status_df[(work_status_df['Start_time']== curr_time) & (work_status_df['End_time']== curr_time_plus30) & (work_status_df['Employee'] not in retained_RUs+retained_RDs)]
    priority_table['RU_Priority'] = priority_table['Remaining_hours_left'].rank(method='min', ascending=False).astype(int)
    priority_table.sort_values(by='RU_Priority', inplace=True)
    priority_table.reset_index(drop=True, inplace=True)
    display(priority_table.head())

    # Pick the new registers for both RU and RD
    print('new_RD_needed_count', new_RD_needed_count)
    new_RU_assigned= priority_table['Employee'][:new_RU_needed_count].tolist()
    new_RD_assigned= priority_table['Employee'][new_RU_needed_count: new_RU_needed_count+new_RD_needed_count].tolist()
    print('new_RU_assigned', new_RU_assigned)
    print('new_RD_assigned', new_RD_assigned)
    
    # Should not break this, rather increment the start time by 30mins so that the while loop continues
    break


Allocating registers between 08:30:00 to 09:00:00 ...


C:\Users\niran\AppData\Local\Temp\ipykernel_28048\1973717794.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_table['RU_Priority'] = priority_table['Remaining_hours_left'].rank(method='min', ascending=False).astype(int)
C:\Users\niran\AppData\Local\Temp\ipykernel_28048\1973717794.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_table.sort_values(by='RU_Priority', inplace=True)


,Start_time,End_time,Employee,Remaining_hours_left,RU_Priority
0,08:30:00,09:00:00,Andrew (AJ) # Bowlen,5.0,1
1,08:30:00,09:00:00,Ricardo Gomez,3.0,2
2,08:30:00,09:00:00,Solymar Kneale,2.5,3
3,08:30:00,09:00:00,Analuisa Flores Teran,2.0,4
4,08:30:00,09:00:00,Finn Broderick,2.0,4


new_RD_needed_count 3
new_RU_assigned ['Andrew (AJ) # Bowlen', 'Ricardo Gomez', 'Solymar Kneale']
new_RD_assigned ['Analuisa Flores Teran', 'Finn Broderick']


In [108]:
extract_list_from_string(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Employee_Assigned_Downstairs'].to_list()[0])

['Mariana Cepeda']

In [106]:
print(type(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Employee_Assigned_Upstairs'].to_list()[0]))
type(extract_list_from_string(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Employee_Assigned_Upstairs'].to_list()[0]))

<class 'str'>


list

In [99]:



extract_list_from_string(greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]['Employee_Assigned_Downstairs'].to_list()[0])

'Mariana Cepeda'

In [97]:
"['Mariana Cepeda', 'AJ John']".replace("'",'').replace("[",'').replace("]",'').split(",")

['Mariana Cepeda', ' AJ John']

In [77]:
[1,2,3] + [4]

[1, 2, 3, 4]

In [54]:
curr_time

datetime.time(7, 30)

In [ ]:
current_RUs= []
current_RDs= []

start_time= 
end_time= 
time= start_time
time_delta= 30mins

while (time < end_time):
    




    time  + = time_delta